# Landing Gear Failure Prediction

In [3]:
import pandas as pd
import numpy as np

# Load datasets (adjust paths if needed)
bogie_data = pd.read_csv('./data/LGD_S_DATASET_1_BogiePitchTrimmer_PressureDrop_OverFlightCycles.csv')
tyre_pressure_data = pd.read_csv('./data/LGD_S_DATASET_2_TyrePressureIndicator_PressureDrop_OverFlightCycles.csv')
temperature_rise_data = pd.read_csv('./data/LGD_S_DATASET_3_TyrePressureIndicator_TemperatureRise_DuringLanding.csv')
brake_temperature_data = pd.read_csv('./data/LGD_S_DATASET_4_LandingGearBrakes_MaxTemperature_OverFlightCycles.csv')
brake_deceleration_data_takeoff = pd.read_csv('./data/LGD_S_DATASET_5_LandingGearBrakes_Deceleration_DuringTakeoff.csv')
brake_deceleration_data_flight = pd.read_csv('./data/LGD_S_DATASET_6_LandingGearBrakes_Deceleration_OverFlightCycles.csv')


In [4]:
# Extract relevant columns for the Bogie Pitch Trimmer dataset
bogie_data = bogie_data[['set_id', 'row_id', 'pressure_1', 'pressure_2', 'temperature_1', 'temperature_2', 'position_1', 'position_2', 'oil_level_1', 'oil_level_2']]

# Split the features and target (e.g., predicting pressure_1 as the failure variable)
X_bogie = bogie_data.drop(columns=['set_id', 'row_id', 'pressure_1'])  # Drop the target column for features
y_bogie = bogie_data['pressure_1']  # Target: 'pressure_1' or any other column related to failure

# Scale features for LSTM (StandardScaler)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_bogie_scaled = scaler.fit_transform(X_bogie)


In [6]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Reshape data for LSTM: (samples, timesteps, features)
X_bogie_scaled = X_bogie_scaled.reshape((X_bogie_scaled.shape[0], 1, X_bogie_scaled.shape[1]))  # Timesteps=1 for each row

# Create and compile the LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=(X_bogie_scaled.shape[1], X_bogie_scaled.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification (failure or not)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_bogie_scaled, y_bogie, epochs=200, batch_size=64, validation_split=0.2, verbose=2)


Epoch 1/200


/Users/frantzmoudoute/anaconda3/envs/jax-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 - 2s - 4ms/step - accuracy: 0.0000e+00 - loss: -2.6771e+04 - val_accuracy: 0.0000e+00 - val_loss: -4.8250e+04
Epoch 2/200
500/500 - 1s - 2ms/step - accuracy: 0.0000e+00 - loss: -6.3590e+04 - val_accuracy: 0.0000e+00 - val_loss: -7.8529e+04
Epoch 3/200
500/500 - 1s - 2ms/step - accuracy: 0.0000e+00 - loss: -9.3045e+04 - val_accuracy: 0.0000e+00 - val_loss: -1.0740e+05
Epoch 4/200
500/500 - 1s - 2ms/step - accuracy: 0.0000e+00 - loss: -1.2173e+05 - val_accuracy: 0.0000e+00 - val_loss: -1.3581e+05
Epoch 5/200
500/500 - 1s - 2ms/step - accuracy: 0.0000e+00 - loss: -1.5006e+05 - val_accuracy: 0.0000e+00 - val_loss: -1.6401e+05
Epoch 6/200
500/500 - 1s - 2ms/step - accuracy: 0.0000e+00 - loss: -1.7807e+05 - val_accuracy: 0.0000e+00 - val_loss: -1.9207e+05
Epoch 7/200
500/500 - 1s - 2ms/step - accuracy: 0.0000e+00 - loss: -2.0611e+05 - val_accuracy: 0.0000e+00 - val_loss: -2.2008e+05
Epoch 8/200
500/500 - 1s - 2ms/step - accuracy: 0.0000e+00 - loss: -2.3438e+05 - val_accuracy: 0.0000e

In [10]:
import xgboost as xgb
import xgboost as xgb

# Prepare the data for XGBoost
X_tyres = tyre_pressure_data.drop(columns=['set_id', 'row_id', 'pressure_1'])  # Drop target column for features
y_tyres = tyre_pressure_data['pressure_1']  # Target column

# Check if there are any missing values and handle them
X_tyres = X_tyres.fillna(X_tyres.mean())
y_tyres = y_tyres.fillna(y_tyres.mean())

# Train XGBoost model
X_dmatrix = xgb.DMatrix(X_tyres, label=y_tyres)
params = {
    'objective': 'reg:squarederror',  # Regression objective for predicting pressure
    'eval_metric': 'rmse',  # Using RMSE for regression
    'max_depth': 5,
    'learning_rate': 0.1,
    'n_estimators': 100
}

xgb_model = xgb.train(params, X_dmatrix, num_boost_round=100)

# Make predictions
y_probs_tyres = xgb_model.predict(X_dmatrix)

# Display predictions
print("Tyre failure probabilities:", y_probs_tyres[:5])



/Users/frantzmoudoute/anaconda3/envs/jax-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [19:46:13] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


Tyre failure probabilities: [243.29605 254.35165 260.32642 252.49661 243.17473]


In [13]:
failure_threshold = 200  # Example threshold, adjust as per domain knowledge

# Classify tyres based on predicted pressure
tyre_failures = y_probs_tyres < failure_threshold  # Boolean array of failure predictions
print("Tyre failure predictions (True = failure, False = normal):", tyre_failures[:10])

# Normalize predicted pressures to a probability scale (0 to 1)
min_pressure = min(y_probs_tyres)
max_pressure = max(y_probs_tyres)
normalized_probs = (y_probs_tyres - min_pressure) / (max_pressure - min_pressure)

# Now `normalized_probs` will be between 0 and 1, indicating relative failure probabilities
print("Normalized Tyre failure probabilities:", normalized_probs[:5])


Tyre failure predictions (True = failure, False = normal): [False False False False False False False False False False]
Normalized Tyre failure probabilities: [0.36748934 0.548976   0.6470566  0.518524   0.36549774]


In [14]:
# Predict failure probabilities for Bogie dataset (using LSTM)
y_probs_bogie = model.predict(X_bogie_scaled)

# Predict failure probabilities for Tyre dataset (using XGBoost)
X_tyres_dmatrix = xgb.DMatrix(X_tyres)
y_probs_tyres = xgb_model.predict(X_tyres_dmatrix)


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 484us/step


In [15]:
# Assuming that all the models give us probabilities of failure for each set_id
# Create DataFrame for predictions

bogie_prob_df = pd.DataFrame({
    'set_id': bogie_data['set_id'],
    'bogie_prob': y_probs_bogie.flatten()
})

tyre_prob_df = pd.DataFrame({
    'set_id': tyre_pressure_data['set_id'],
    'tyre_prob': y_probs_tyres
})

# Merge the predictions from both models
combined_prob_df = pd.merge(bogie_prob_df, tyre_prob_df, on='set_id', how='outer')

# Combine the probabilities to create a final health indicator (you can use a weighted average if needed)
combined_prob_df['health_indicator'] = (combined_prob_df['bogie_prob'] + combined_prob_df['tyre_prob']) / 2

# Rank the health of each system
combined_prob_df['health_rank'] = combined_prob_df['health_indicator'].rank(ascending=False)

# Display the final health ranking
print(combined_prob_df[['set_id', 'health_indicator', 'health_rank']].head())


   set_id  health_indicator  health_rank
0     0.0        122.148026    1713500.5
1     0.0        127.675827     861900.5
2     0.0        130.663208     469100.5
3     0.0        126.748306    1027900.5
4     0.0        122.087364    1720700.5
